In [1]:
from detectron2.config import get_cfg
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data import MetadataCatalog, DatasetCatalog
import detectron2.data.transforms as T
from detectron2.structures.image_list import ImageList
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.modeling.box_regression import Box2BoxTransform
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetMapper
from detectron2.data import (
    build_detection_test_loader,
    build_detection_train_loader,
)
from detectron2.data.samplers import TrainingSampler

from fsdet.modeling.roi_heads.fast_rcnn import FastRCNNOutputs

import cv2
import torch, torchvision
import logging
logging.basicConfig(level=logging.INFO)

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



# Register my_dataset

In [2]:
train = True
if train == True:
    json_dir = 'datasets/coco_experiments/seed1/full_box_30shot_person_trainval.json'
    image_dir = 'datasets/coco/trainval2014'
    register_coco_instances("my_dataset", {}, json_dir, image_dir)
    # MetadataCatalog.get("my_dataset").thing_classes = ["person"] # json file itself has all the category metadata
else:
    json_dir = 'datasets/coco_experiments/seed1/full_box_1000shot_person_test.json'
    image_dir = 'datasets/coco/trainval2014'
    register_coco_instances("my_dataset", {}, json_dir, image_dir)

# Build model using cfg file

In [3]:
cfg = get_cfg()
cfg.merge_from_file('configs/COCO-detection/faster_rcnn_R_101_FPN_base.yaml')
model = build_model(cfg)  # returns a torch.nn.Module

model.eval()
metadata = MetadataCatalog.get(cfg.DATASETS.TEST[0])

ckpt_file = 'checkpoints/coco/base_model/model_final.pth'
DetectionCheckpointer(model).load(ckpt_file)

INFO:fvcore.common.checkpoint:Loading checkpoint from checkpoints/coco/base_model/model_final.pth


{'optimizer': {'state': {140508395095672: {'momentum_buffer': tensor([[[[-1.2293e-04]],
    
             [[-3.7314e-04]],
    
             [[-3.6808e-04]],
    
             ...,
    
             [[-2.4900e-04]],
    
             [[-1.9884e-04]],
    
             [[ 3.8510e-05]]],
    
    
            [[[ 2.9357e-04]],
    
             [[ 1.2172e-03]],
    
             [[ 1.1192e-03]],
    
             ...,
    
             [[ 8.4696e-04]],
    
             [[ 1.7627e-03]],
    
             [[ 1.2127e-03]]],
    
    
            [[[-1.2993e-04]],
    
             [[ 3.6365e-04]],
    
             [[ 1.1718e-04]],
    
             ...,
    
             [[ 1.7940e-04]],
    
             [[ 3.9368e-04]],
    
             [[-1.0029e-04]]],
    
    
            ...,
    
    
            [[[-3.4145e-04]],
    
             [[-3.4512e-04]],
    
             [[-1.2064e-04]],
    
             ...,
    
             [[-1.1182e-04]],
    
             [[-1.0942e-03]],
    


In [4]:
print(cfg)

CUDNN_BENCHMARK: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  FILTER_EMPTY_ANNOTATIONS: True
  NUM_WORKERS: 4
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: ()
  PROPOSAL_FILES_TRAIN: ()
  TEST: ('coco_test_base',)
  TRAIN: ('coco_trainval_base',)
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: False
    SIZE: [0.9, 0.9]
    TYPE: relative_range
  FORMAT: BGR
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (640, 672, 704, 736, 768, 800)
  MIN_SIZE_TRAIN_SAMPLING: choice
  RANDOM_FLIP: horizontal
MODEL:
  ANCHOR_GENERATOR:
    ANGLES: [[-90, 0, 90]]
    ASPECT_RATIOS: [[0.5, 1.0, 2.0]]
    NAME: DefaultAnchorGenerator
    OFFSET: 0.0
    SIZES: [[32], [64], [128], [256], [512]]
  BACKBONE:
    FREEZE_AT: 2
    NAME: build_resnet_fpn_backbone
  DEVICE: cuda
  FPN:
    FUSE_TYPE: sum
    IN_FEATURES: ['res

# Load data from dataset

In [5]:
data_list = DatasetCatalog.get("my_dataset")
print(len(data_list))
# print(data_list[0])

INFO:detectron2.data.datasets.coco:Loaded 239 images in COCO format from datasets/coco_experiments/seed1/full_box_1000shot_person_test.json


239


In [6]:
transform_gen = T.ResizeShortestEdge(
            [cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST],
            cfg.INPUT.MAX_SIZE_TEST,
            )
data_loader = build_detection_train_loader(DatasetCatalog.get("my_dataset"),
                                           mapper=DatasetMapper(cfg, is_train=True, augmentations=[transform_gen]),
                                           total_batch_size = 10)
data_loader_it = iter(data_loader)
data = next(data_loader_it)
print(len(data))
print(data[0]['image'].shape)
# print(data)

INFO:detectron2.data.datasets.coco:Loaded 239 images in COCO format from datasets/coco_experiments/seed1/full_box_1000shot_person_test.json
INFO:detectron2.data.dataset_mapper:[DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=[800, 800], max_size=1333)]
INFO:detectron2.data.common:Serializing 239 elements to byte tensors and concatenating them all ...
INFO:detectron2.data.common:Serialized dataset takes 0.67 MiB


10
torch.Size([3, 800, 1067])


# Forward the model until box_head and get the feature vectors

In [7]:
img_tensors_list = [d['image'].float() for d in data]
# images = ImageList.from_tensors([image, image2]).to('cuda')
images = ImageList.from_tensors(img_tensors_list, model.backbone.size_divisibility).to('cuda')
input_images = images.tensor
print(input_images.shape)

torch.Size([10, 3, 800, 1344])


In [8]:
features = model.backbone(input_images)
print(features.keys(), '\n')
# FPN's multiple feature map output

proposals_list, _ = model.proposal_generator(images, features)
print(proposals_list[0].proposal_boxes)
print(proposals_list[0].objectness_logits.shape, '\n')
# RPN's proposals outputs, including the bboxes of 1000 proposals and their objectness_logits

# from detectron2.modeling.proposal_generator.proposal_utils import add_ground_truth_to_proposals
# targets = [d['instances'] for d in data]
# gt_boxes = [x.gt_boxes.to('cuda') for x in targets]

# proposal_append_gt = True
# if proposal_append_gt:
#     proposals_list = add_ground_truth_to_proposals(gt_boxes, proposals_list)

instances, _ = model.roi_heads(images, features, proposals_list)
print(instances[0], '\n')
# print(instances[1], '\n')
# The final outputs of the classifier and box regressor, inclduing the pred_boxes, their scores and classes

dict_keys(['p2', 'p3', 'p4', 'p5', 'p6']) 

Boxes(tensor([[ 597.2145,  543.9638,  640.3090,  598.3193],
        [ 304.7969,  239.5787,  386.2406,  272.7734],
        [ 596.1298,  546.2780,  638.2923,  597.6371],
        ...,
        [ 491.4433,  290.8164, 1067.0000,  800.0000],
        [ 385.6718,  453.7903,  515.6587,  564.5177],
        [ 415.0699,  378.2701,  507.3828,  519.9290]], device='cuda:0'))
torch.Size([1000]) 

Instances(num_instances=23, image_height=800, image_width=1067, fields=[pred_boxes: Boxes(tensor([[597.4456, 544.1341, 637.7045, 599.6450],
        [240.2511, 270.9893, 292.3272, 295.0779],
        [506.3135, 513.2130, 590.6134, 604.4691],
        [241.7917, 270.7968, 294.6407, 293.9287],
        [242.1823, 271.1031, 294.1516, 293.9632],
        [582.1412, 447.5813, 697.4754, 549.2993],
        [597.9471, 541.3864, 637.6290, 619.6011],
        [583.8041, 424.8033, 696.0548, 542.1043],
        [308.9313, 268.2355, 347.5604, 300.5369],
        [625.1008, 479.5725, 658.

/home/cloudlet/anaconda3/envs/fsdet/lib/python3.6/site-packages/detectron2/modeling/roi_heads/fast_rcnn.py:124: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370116979/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  filter_inds = filter_mask.nonzero()


In [9]:
box_features = model.roi_heads.box_pooler(
    [features[f] for f in ["p2", "p3", "p4", "p5"]], [x.proposal_boxes for x in proposals_list]
)
box_features = model.roi_heads.box_head(box_features).to('cpu')
print(box_features.shape)
# print(box_features[:3, :20])
# print(box_features[1000:1003, :20])

torch.Size([10000, 1024])


# Sample proposals X and get their groundtruth label y

In [10]:
from detectron2.modeling.proposal_generator.proposal_utils import add_ground_truth_to_proposals
from detectron2.modeling.sampling import subsample_labels
from detectron2.structures import Boxes, Instances, pairwise_iou
from detectron2.modeling.matcher import Matcher
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 100
cfg.MODEL.ROI_HEADS.POSITIVE_FRACTION = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
def _sample_proposals(matched_idxs, matched_labels, gt_classes):
    """
    Based on the matching between N proposals and M groundtruth,
    sample the proposals and set their classification labels.
    Args:
        matched_idxs (Tensor): a vector of length N, each is the best-matched
            gt index in [0, M) for each proposal.
        matched_labels (Tensor): a vector of length N, the matcher's label
            (one of cfg.MODEL.ROI_HEADS.IOU_LABELS) for each proposal.
        gt_classes (Tensor): a vector of length M.
    Returns:
        Tensor: a vector of indices of sampled proposals. Each is in [0, N).
        Tensor: a vector of the same length, the classification label for
            each sampled proposal. Each sample is labeled as either a category in
            [0, num_classes) or the background (num_classes).
    """
    has_gt = gt_classes.numel() > 0
    # Get the corresponding GT for each proposal
    if has_gt:
        gt_classes = gt_classes[matched_idxs]
        # Label unmatched proposals (0 label from matcher) as background (label=num_classes)
        gt_classes[matched_labels == 0] = cfg.MODEL.ROI_HEADS.NUM_CLASSES
        # Label ignore proposals (-1 label)
        gt_classes[matched_labels == -1] = -1
    else:
        gt_classes = torch.zeros_like(matched_idxs) + cfg.MODEL.ROI_HEADS.NUM_CLASSES

    sampled_fg_idxs, sampled_bg_idxs = subsample_labels(
        gt_classes,
        cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE,
        cfg.MODEL.ROI_HEADS.POSITIVE_FRACTION,
        cfg.MODEL.ROI_HEADS.NUM_CLASSES,
    )

    sampled_idxs = torch.cat([sampled_fg_idxs, sampled_bg_idxs], dim=0)
    return sampled_idxs, gt_classes[sampled_idxs]

In [11]:
proposals = [proposal.to('cpu') for proposal in proposals_list]
print(len(proposals), type(proposals[0]))
# print(proposals[0])
targets = [d['instances'] for d in data]
print(len(targets), type(targets[0]))
print(targets[0])

10 <class 'detectron2.structures.instances.Instances'>
10 <class 'detectron2.structures.instances.Instances'>
Instances(num_instances=5, image_height=800, image_width=1067, fields=[gt_boxes: Boxes(tensor([[ 526.7812,   17.2833,  791.8307,  365.7166],
        [ 794.8483,   55.7333, 1067.0000,  442.2500],
        [ 592.1183,  205.5667, 1067.0000,  800.0000],
        [   0.0000,  108.8667,  330.3032,  800.0000],
        [  62.9363,   21.5667,  267.9504,  350.5667]])), gt_classes: tensor([0, 0, 0, 0, 0])])


In [12]:

proposal_matcher = Matcher(
            cfg.MODEL.ROI_HEADS.IOU_THRESHOLDS,
            cfg.MODEL.ROI_HEADS.IOU_LABELS,
            allow_low_quality_matches=False,
        )

gt_boxes = [x.gt_boxes for x in targets]
proposals_with_gt = []
num_fg_samples = []
num_bg_samples = []
sampled_ft_vecs_list = []

proposal_append_gt = False
if proposal_append_gt:
    proposals = add_ground_truth_to_proposals(gt_boxes, proposals)
    

for i, (proposals_per_image, targets_per_image) in enumerate(zip(proposals, targets)):
    feature_vectors =box_features[i*1000:(i+1)*1000, :]
    has_gt = len(targets_per_image) > 0
    match_quality_matrix = pairwise_iou(
        targets_per_image.gt_boxes, proposals_per_image.proposal_boxes
    )
    matched_idxs, matched_labels = proposal_matcher(match_quality_matrix)
    sampled_idxs, gt_classes = _sample_proposals(
        matched_idxs, matched_labels, targets_per_image.gt_classes
    )

    # Set target attributes of the sampled proposals:
    feature_vectors = feature_vectors[sampled_idxs]
    proposals_per_image = proposals_per_image[sampled_idxs]
    proposals_per_image.gt_classes = gt_classes
    
    # We index all the attributes of targets that start with "gt_"
    # and have not been added to proposals yet (="gt_classes").
    if has_gt:
        sampled_targets = matched_idxs[sampled_idxs]
        # NOTE: here the indexing waste some compute, because heads
        # will filter the proposals again (by foreground/background,
        # etc), so we essentially index the data twice.
        for (
            trg_name,
            trg_value,
        ) in targets_per_image.get_fields().items():
            if trg_name.startswith(
                "gt_"
            ) and not proposals_per_image.has(trg_name):
                proposals_per_image.set(
                    trg_name, trg_value[sampled_targets]
                )
    else:
        gt_boxes = Boxes(
            targets_per_image.gt_boxes.tensor.new_zeros(
                (len(sampled_idxs), 4)
            )
        )
        proposals_per_image.gt_boxes = gt_boxes

    num_bg_samples.append(
        (gt_classes == cfg.MODEL.ROI_HEADS.NUM_CLASSES).sum().item()
    )
    num_fg_samples.append(gt_classes.numel() - num_bg_samples[-1])
    sampled_ft_vecs_list.append(feature_vectors)
    proposals_with_gt.append(proposals_per_image)
            
print(match_quality_matrix.shape)
print(matched_idxs.shape, matched_labels.shape)
print(sampled_idxs.shape, gt_classes.shape)
print(len(proposals_with_gt))
print(sampled_ft_vecs_list[0].shape)

torch.Size([1, 1000])
torch.Size([1000]) torch.Size([1000])
torch.Size([100]) torch.Size([100])
10
torch.Size([100, 1024])


# Fit SVM

In [13]:
from sklearn import svm
from joblib import dump, load

In [14]:
X = torch.vstack(sampled_ft_vecs_list).detach().numpy()
y = torch.vstack([p.gt_classes.reshape((-1, 1)) for p in proposals_with_gt]).detach().numpy().ravel()
# print(X.shape)
# print(y.shape)
from sklearn import svm
# clf = svm.SVC()
# clf.fit(X, y)

In [15]:
# from joblib import dump, load
# dump(clf, 'svm_model.joblib') 
# clf2 = load('svm_model.joblib') 

# Evaluate the RCNN+SVM detector

In [16]:
clf = load('svm_model.joblib') 

In [17]:
y_hat = clf.predict(X)

In [18]:
print(y.shape)
print(y_hat.shape)

(1000,)
(1000,)


In [19]:
print(y[:50])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [20]:
print(y_hat[:50])

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [21]:
print(float((y_hat != y).mean()))

0.225
